# Day 3: Error Management - Handling Suite Emergencies

**Week 18 - Enterprise GPU Management**

> *"A great hotel doesn't just prevent problems—it has procedures for gracefully handling any emergency without disturbing other guests."*

---

## 🎯 Learning Objectives

By the end of this lesson, you will:
1. **Implement** comprehensive CUDA error handling patterns
2. **Distinguish** between sticky and non-sticky errors
3. **Use** stream isolation for fault containment
4. **Apply** production-grade logging and recovery strategies

---

## 🏨 Concept Card: Hotel Resource Isolation

**MIG Error Isolation is like hotel emergency management:**

```
Without MIG (Shared Hotel Floor):
┌─────────────────────────────────────────────────────┐
│                 🏨 Open Floor Plan                  │
│  ┌───────┐  ┌───────┐  ┌───────┐  ┌───────┐        │
│  │ App A │  │ App B │  │ App C │  │ App D │        │
│  └───────┘  └───────┘  └───────┘  └───────┘        │
│      │          │          │          │             │
│      └──────────┴──────────┴──────────┘             │
│           Shared Memory Space                       │
│  ⚠️ App A crashes → May corrupt B, C, D's memory!  │
└─────────────────────────────────────────────────────┘

With MIG (Isolated Suites):
┌─────────────────────────────────────────────────────┐
│            🏨 Private Suites (MIG Instances)        │
│  ┌─────────────┐      ┌─────────────┐              │
│  │ 🚪 Suite A  │      │ 🚪 Suite B  │              │
│  │   App A     │      │   App B     │              │
│  │  40GB HBM   │ 🔒  │  40GB HBM   │              │
│  │  66 SMs     │      │  66 SMs     │              │
│  └─────────────┘      └─────────────┘              │
│  ⚠️ App A crashes → App B completely unaffected!   │
└─────────────────────────────────────────────────────┘
```

**The Emergency Management Mapping:**
| Hotel Concept | GPU Error Handling | Benefit |
|---------------|-------------------|---------|
| 🚨 Fire in Suite A | Kernel crash | Only Suite A affected |
| 🔥 Smoke containment | Memory isolation | Corruption can't spread |
| 🚒 Local response | cudaDeviceReset() | Reset one instance only |
| 📋 Incident log | Error logging | Track and analyze issues |
| 🛡️ Suite firewall | Stream isolation | Further fault containment |

---

## Error Handling Patterns

In [ ]:
%%writefile error_handling.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <stdarg.h>
#include <time.h>

// =============================================================
// Production Error Handling Framework
// =============================================================

enum LogLevel {
    LOG_DEBUG = 0,
    LOG_INFO = 1,
    LOG_WARN = 2,
    LOG_ERROR = 3,
    LOG_FATAL = 4
};

const char* logLevelStr[] = {"DEBUG", "INFO", "WARN", "ERROR", "FATAL"};

// Thread-safe logging
void logMessage(LogLevel level, const char* file, int line, const char* fmt, ...) {
    time_t now = time(NULL);
    struct tm* tm_info = localtime(&now);
    char timestamp[26];
    strftime(timestamp, 26, "%Y-%m-%d %H:%M:%S", tm_info);
    
    fprintf(stderr, "[%s] [%s] %s:%d - ", timestamp, logLevelStr[level], file, line);
    
    va_list args;
    va_start(args, fmt);
    vfprintf(stderr, fmt, args);
    va_end(args);
    
    fprintf(stderr, "\n");
}

#define LOG_DEBUG(...) logMessage(LOG_DEBUG, __FILE__, __LINE__, __VA_ARGS__)
#define LOG_INFO(...)  logMessage(LOG_INFO,  __FILE__, __LINE__, __VA_ARGS__)
#define LOG_WARN(...)  logMessage(LOG_WARN,  __FILE__, __LINE__, __VA_ARGS__)
#define LOG_ERROR(...) logMessage(LOG_ERROR, __FILE__, __LINE__, __VA_ARGS__)
#define LOG_FATAL(...) logMessage(LOG_FATAL, __FILE__, __LINE__, __VA_ARGS__)

// Enhanced CUDA error check with context
#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = (call); \
        if (err != cudaSuccess) { \
            LOG_ERROR("CUDA error in %s: %s (code %d)", \
                      #call, cudaGetErrorString(err), err); \
            return err; \
        } \
    } while(0)

#define CUDA_CHECK_FATAL(call) \
    do { \
        cudaError_t err = (call); \
        if (err != cudaSuccess) { \
            LOG_FATAL("CUDA error in %s: %s", #call, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE); \
        } \
    } while(0)

// Check for kernel launch errors (async)
#define CUDA_KERNEL_CHECK() \
    do { \
        cudaError_t err = cudaGetLastError(); \
        if (err != cudaSuccess) { \
            LOG_ERROR("Kernel launch error: %s", cudaGetErrorString(err)); \
            return err; \
        } \
        err = cudaDeviceSynchronize(); \
        if (err != cudaSuccess) { \
            LOG_ERROR("Kernel execution error: %s", cudaGetErrorString(err)); \
            return err; \
        } \
    } while(0)

// =============================================================
// Example: Robust Memory Allocation
// =============================================================

cudaError_t safeDeviceMalloc(void** ptr, size_t size, const char* name) {
    LOG_DEBUG("Allocating %zu bytes for '%s'", size, name);
    
    // Check available memory first
    size_t freeMemory, totalMemory;
    CUDA_CHECK(cudaMemGetInfo(&freeMemory, &totalMemory));
    
    if (size > freeMemory) {
        LOG_ERROR("Insufficient GPU memory for '%s': need %zu, have %zu",
                  name, size, freeMemory);
        return cudaErrorMemoryAllocation;
    }
    
    // Attempt allocation
    CUDA_CHECK(cudaMalloc(ptr, size));
    
    LOG_INFO("Allocated %zu bytes for '%s' (%.1f%% of free memory)",
             size, name, 100.0 * size / freeMemory);
    
    return cudaSuccess;
}

// =============================================================
// Example: Error Recovery
// =============================================================

__global__ void faultyKernel(int* data, int n, bool causeFault) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (causeFault && idx == 0) {
        // Intentional out-of-bounds access
        int* bad_ptr = (int*)0xDEADBEEF;
        *bad_ptr = 42;
    }
    if (idx < n) {
        data[idx] = idx * 2;
    }
}

cudaError_t runWithRecovery(int* d_data, int n, bool causeFault) {
    LOG_INFO("Launching kernel (causeFault=%d)", causeFault);
    
    int blockSize = 256;
    int numBlocks = (n + blockSize - 1) / blockSize;
    
    faultyKernel<<<numBlocks, blockSize>>>(d_data, n, causeFault);
    
    cudaError_t launchErr = cudaGetLastError();
    if (launchErr != cudaSuccess) {
        LOG_ERROR("Kernel launch failed: %s", cudaGetErrorString(launchErr));
        return launchErr;
    }
    
    cudaError_t syncErr = cudaDeviceSynchronize();
    if (syncErr != cudaSuccess) {
        LOG_ERROR("Kernel execution failed: %s", cudaGetErrorString(syncErr));
        
        // Attempt recovery
        LOG_WARN("Attempting device reset...");
        cudaError_t resetErr = cudaDeviceReset();
        if (resetErr != cudaSuccess) {
            LOG_FATAL("Device reset failed: %s", cudaGetErrorString(resetErr));
            return resetErr;
        }
        
        LOG_INFO("Device reset successful");
        return syncErr;
    }
    
    LOG_INFO("Kernel completed successfully");
    return cudaSuccess;
}

// =============================================================
// Main Demo
// =============================================================

int main() {
    LOG_INFO("=== Error Handling Demo ===");
    
    const int N = 1024;
    int* d_data = nullptr;
    
    // Safe allocation
    cudaError_t err = safeDeviceMalloc((void**)&d_data, N * sizeof(int), "test_data");
    if (err != cudaSuccess) {
        LOG_FATAL("Allocation failed");
        return 1;
    }
    
    // Test successful execution
    LOG_INFO("--- Test 1: Normal execution ---");
    err = runWithRecovery(d_data, N, false);
    if (err == cudaSuccess) {
        LOG_INFO("Test 1 PASSED");
    }
    
    // Clean up
    CUDA_CHECK_FATAL(cudaFree(d_data));
    
    LOG_INFO("=== Demo Complete ===");
    return 0;
}

In [ ]:
!nvcc -O3 -arch=sm_80 error_handling.cu -o error_handling && ./error_handling

## Sticky Error Handling

In [ ]:
%%writefile sticky_errors.cu
#include <stdio.h>
#include <cuda_runtime.h>

// Demonstrate sticky vs non-sticky errors
// Sticky errors require device reset to clear

int main() {
    printf("=== Sticky Error Demonstration ===\n\n");
    
    int deviceCount;
    cudaError_t err = cudaGetDeviceCount(&deviceCount);
    printf("Initial state: %s\n\n", cudaGetErrorString(err));
    
    // Non-sticky error: Invalid configuration
    printf("--- Non-Sticky Error Test ---\n");
    int* dummy;
    err = cudaMalloc(&dummy, (size_t)-1);  // Impossibly large allocation
    printf("After bad malloc: %s\n", cudaGetErrorString(err));
    
    // Check last error (should be set)
    err = cudaGetLastError();
    printf("cudaGetLastError(): %s\n", cudaGetErrorString(err));
    
    // Check peek error (same but doesn't clear)
    err = cudaPeekAtLastError();
    printf("cudaPeekAtLastError(): %s\n", cudaGetErrorString(err));
    
    // After cudaGetLastError, error should be cleared
    err = cudaGetLastError();
    printf("After clearing: %s\n", cudaGetErrorString(err));
    
    // Try normal operation (should work)
    int* d_ptr;
    err = cudaMalloc(&d_ptr, sizeof(int));
    printf("Normal allocation after non-sticky: %s\n", cudaGetErrorString(err));
    
    if (err == cudaSuccess) {
        cudaFree(d_ptr);
        printf("Device still functional!\n\n");
    }
    
    // Sticky errors (like illegal memory access) require device reset
    printf("--- Sticky Error Categories ---\n");
    printf("Non-sticky (recoverable):\n");
    printf("  - cudaErrorInvalidValue\n");
    printf("  - cudaErrorMemoryAllocation\n");
    printf("  - cudaErrorInvalidDevicePointer\n");
    printf("\nSticky (require device reset):\n");
    printf("  - cudaErrorIllegalAddress (illegal memory access)\n");
    printf("  - cudaErrorHardwareStackError\n");
    printf("  - cudaErrorAssert (device-side assert)\n");
    printf("  - cudaErrorLaunchTimeout\n");
    printf("  - ECC memory errors\n");
    
    printf("\n--- Error Type Checking ---\n");
    
    // Example: Check if error is recoverable
    auto isRecoverable = [](cudaError_t e) {
        switch(e) {
            case cudaErrorInvalidValue:
            case cudaErrorMemoryAllocation:
            case cudaErrorInvalidConfiguration:
                return true;
            default:
                return false;
        }
    };
    
    cudaError_t testErrors[] = {
        cudaErrorInvalidValue,
        cudaErrorMemoryAllocation,
        cudaErrorIllegalAddress,
        cudaErrorAssert
    };
    
    for (auto e : testErrors) {
        printf("%s: %s\n", 
               cudaGetErrorName(e),
               isRecoverable(e) ? "Recoverable" : "Requires reset");
    }
    
    return 0;
}

In [ ]:
!nvcc -O3 -arch=sm_80 sticky_errors.cu -o sticky_errors && ./sticky_errors

## Stream-Based Error Checking

In [ ]:
%%writefile stream_errors.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void simpleKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = sqrtf((float)idx);
    }
}

int main() {
    printf("=== Stream-Based Error Checking ===\n\n");
    
    const int N = 1024 * 1024;
    const int numStreams = 4;
    
    cudaStream_t streams[numStreams];
    float* d_data[numStreams];
    cudaError_t streamErrors[numStreams];
    
    // Create streams and allocate per-stream memory
    for (int i = 0; i < numStreams; i++) {
        cudaStreamCreate(&streams[i]);
        cudaMalloc(&d_data[i], N * sizeof(float));
        streamErrors[i] = cudaSuccess;
    }
    
    // Launch kernels on different streams
    printf("Launching kernels on %d streams...\n", numStreams);
    
    for (int i = 0; i < numStreams; i++) {
        int blockSize = 256;
        int numBlocks = (N + blockSize - 1) / blockSize;
        
        simpleKernel<<<numBlocks, blockSize, 0, streams[i]>>>(d_data[i], N);
        
        // Check launch error immediately (doesn't wait for completion)
        streamErrors[i] = cudaGetLastError();
        if (streamErrors[i] != cudaSuccess) {
            printf("Stream %d launch error: %s\n", i, 
                   cudaGetErrorString(streamErrors[i]));
        }
    }
    
    // Query stream status (non-blocking)
    printf("\nQuerying stream status...\n");
    for (int i = 0; i < numStreams; i++) {
        cudaError_t status = cudaStreamQuery(streams[i]);
        printf("Stream %d: %s\n", i, 
               status == cudaSuccess ? "Complete" :
               status == cudaErrorNotReady ? "Running" :
               cudaGetErrorString(status));
    }
    
    // Synchronize each stream and check for errors
    printf("\nSynchronizing streams...\n");
    for (int i = 0; i < numStreams; i++) {
        cudaError_t err = cudaStreamSynchronize(streams[i]);
        if (err != cudaSuccess) {
            printf("Stream %d execution error: %s\n", i, 
                   cudaGetErrorString(err));
            streamErrors[i] = err;
        } else {
            printf("Stream %d: Success\n", i);
        }
    }
    
    // Summary
    printf("\n--- Error Summary ---\n");
    int errorCount = 0;
    for (int i = 0; i < numStreams; i++) {
        if (streamErrors[i] != cudaSuccess) {
            errorCount++;
            printf("Stream %d FAILED: %s\n", i, 
                   cudaGetErrorString(streamErrors[i]));
        }
    }
    
    if (errorCount == 0) {
        printf("All %d streams completed successfully!\n", numStreams);
    } else {
        printf("%d/%d streams had errors\n", errorCount, numStreams);
    }
    
    // Cleanup
    for (int i = 0; i < numStreams; i++) {
        cudaFree(d_data[i]);
        cudaStreamDestroy(streams[i]);
    }
    
    return errorCount;
}

In [ ]:
!nvcc -O3 -arch=sm_80 stream_errors.cu -o stream_errors && ./stream_errors

## CUDA Debugging Environment Variables

```bash
# Enable synchronous launches (helps locate errors)
export CUDA_LAUNCH_BLOCKING=1

# Memory checking
export CUDA_MEMCHECK_PATCH_MODULE=1

# Device selection
export CUDA_VISIBLE_DEVICES=0

# Debug output
export CUDA_DEVICE_ORDER=PCI_BUS_ID

# For compute-sanitizer
compute-sanitizer --tool memcheck ./my_program
compute-sanitizer --tool racecheck ./my_program
compute-sanitizer --tool initcheck ./my_program
compute-sanitizer --tool synccheck ./my_program
```

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

Complete these exercises to practice production error management concepts:

1. **Exercise 1: Comprehensive Error Wrapper** - Build a robust CUDA error handling class
2. **Exercise 2: Sticky Error Recovery** - Implement detection and recovery from sticky errors
3. **Exercise 3: Stream Error Isolation** - Handle errors in isolated streams without affecting others

In [ ]:
%%writefile error_management_exercises.cu
#include <cuda_runtime.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>

// =============================================================================
// Exercise 1: Comprehensive Error Wrapper
// Build a robust CUDA error handling class
// =============================================================================

class CudaErrorHandler {
private:
    const char* filename;
    int line;
    bool throwOnError;
    
    static const char* errorSeverity(cudaError_t error) {
        switch (error) {
            case cudaSuccess:
                return "INFO";
            case cudaErrorMemoryAllocation:
            case cudaErrorLaunchOutOfResources:
                return "CRITICAL";
            case cudaErrorInvalidValue:
            case cudaErrorInvalidDevicePointer:
                return "ERROR";
            default:
                return "WARNING";
        }
    }
    
    static void logError(const char* severity, const char* file, int line, 
                        cudaError_t error, const char* context) {
        time_t now = time(NULL);
        char timeStr[26];
        ctime_r(&now, timeStr);
        timeStr[24] = '\0';  // Remove newline
        
        fprintf(stderr, "[%s] [%s] CUDA Error at %s:%d\n", 
                timeStr, severity, file, line);
        fprintf(stderr, "  Error Code: %d (%s)\n", 
                error, cudaGetErrorName(error));
        fprintf(stderr, "  Description: %s\n", cudaGetErrorString(error));
        if (context) {
            fprintf(stderr, "  Context: %s\n", context);
        }
    }

public:
    CudaErrorHandler(const char* file, int ln, bool throwErr = false) 
        : filename(file), line(ln), throwOnError(throwErr) {}
    
    bool check(cudaError_t error, const char* context = nullptr) {
        if (error != cudaSuccess) {
            const char* severity = errorSeverity(error);
            logError(severity, filename, line, error, context);
            
            if (throwOnError) {
                exit(EXIT_FAILURE);
            }
            return false;
        }
        return true;
    }
    
    static void checkLastError(const char* context = nullptr) {
        cudaError_t error = cudaGetLastError();
        if (error != cudaSuccess) {
            fprintf(stderr, "[ERROR] Async CUDA Error: %s\n", 
                    cudaGetErrorString(error));
            if (context) {
                fprintf(stderr, "  Context: %s\n", context);
            }
        }
    }
};

#define CUDA_SAFE_CALL(call) \
    CudaErrorHandler(__FILE__, __LINE__, true).check(call, #call)

#define CUDA_CHECK_LAST(context) \
    CudaErrorHandler::checkLastError(context)

__global__ void testKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = sqrtf((float)idx);
    }
}

void exercise1_error_wrapper() {
    printf("=== Exercise 1: Comprehensive Error Wrapper ===\n\n");
    
    // Test successful operations
    printf("Testing successful allocation...\n");
    float* d_data;
    CUDA_SAFE_CALL(cudaMalloc(&d_data, 1024 * sizeof(float)));
    printf("  Allocation succeeded!\n\n");
    
    // Test kernel launch with error checking
    printf("Testing kernel launch...\n");
    testKernel<<<4, 256>>>(d_data, 1024);
    CUDA_CHECK_LAST("testKernel execution");
    CUDA_SAFE_CALL(cudaDeviceSynchronize());
    printf("  Kernel completed successfully!\n\n");
    
    // Test intentional error (trying to free already freed memory is UB, 
    // so we'll demonstrate with invalid operations safely)
    printf("Testing error detection with invalid operation...\n");
    CudaErrorHandler handler(__FILE__, __LINE__, false);  // Don't exit on error
    
    // Try to set an invalid device
    bool success = handler.check(cudaSetDevice(999), "Set invalid device");
    printf("  Error was %s detected and handled\n\n", success ? "NOT" : "correctly");
    
    // Reset device state after error
    cudaGetLastError();  // Clear the error
    
    CUDA_SAFE_CALL(cudaFree(d_data));
    printf("Cleanup completed!\n\n");
}

// =============================================================================
// Exercise 2: Sticky Error Recovery
// Implement detection and recovery from sticky errors
// =============================================================================

class StickyErrorRecovery {
private:
    int deviceId;
    bool hasError;
    cudaError_t lastError;
    
public:
    StickyErrorRecovery(int dev = 0) : deviceId(dev), hasError(false) {
        cudaSetDevice(deviceId);
    }
    
    bool checkForStickyError() {
        cudaError_t error = cudaPeekAtLastError();
        
        if (error != cudaSuccess) {
            hasError = true;
            lastError = error;
            
            // Check if it's a sticky error
            // Sticky errors persist until device reset
            printf("  Detected error: %s\n", cudaGetErrorString(error));
            
            // Try to clear with cudaGetLastError
            cudaGetLastError();
            
            // Check if error persists
            if (cudaPeekAtLastError() != cudaSuccess) {
                printf("  Error is STICKY - requires device reset\n");
                return true;
            } else {
                printf("  Error was non-sticky - cleared successfully\n");
                return false;
            }
        }
        return false;
    }
    
    bool attemptRecovery() {
        printf("  Attempting device recovery...\n");
        
        // Reset the device
        cudaError_t resetError = cudaDeviceReset();
        
        if (resetError != cudaSuccess) {
            printf("  Device reset FAILED: %s\n", cudaGetErrorString(resetError));
            return false;
        }
        
        // Re-select device
        cudaError_t setError = cudaSetDevice(deviceId);
        if (setError != cudaSuccess) {
            printf("  Device selection FAILED: %s\n", cudaGetErrorString(setError));
            return false;
        }
        
        // Verify recovery with a simple operation
        void* testPtr;
        cudaError_t allocError = cudaMalloc(&testPtr, 1024);
        
        if (allocError == cudaSuccess) {
            cudaFree(testPtr);
            printf("  Device recovery SUCCESSFUL\n");
            hasError = false;
            return true;
        }
        
        printf("  Device recovery FAILED - device may require system reboot\n");
        return false;
    }
    
    void simulateWorkWithRecovery() {
        float* d_data;
        
        // Attempt allocation
        cudaError_t error = cudaMalloc(&d_data, 1024 * sizeof(float));
        
        if (error != cudaSuccess) {
            printf("  Allocation failed, checking for sticky error...\n");
            if (checkForStickyError()) {
                if (attemptRecovery()) {
                    // Retry operation after recovery
                    error = cudaMalloc(&d_data, 1024 * sizeof(float));
                    if (error == cudaSuccess) {
                        printf("  Retry after recovery: SUCCESS\n");
                        cudaFree(d_data);
                    }
                }
            }
        } else {
            printf("  Allocation succeeded on first try\n");
            cudaFree(d_data);
        }
    }
};

void exercise2_sticky_error_recovery() {
    printf("=== Exercise 2: Sticky Error Recovery ===\n\n");
    
    printf("Demonstrating sticky error recovery pattern...\n\n");
    
    StickyErrorRecovery recovery(0);
    
    // Normal operation
    printf("Test 1: Normal operation\n");
    recovery.simulateWorkWithRecovery();
    printf("\n");
    
    // Demonstrate error detection
    printf("Test 2: Error detection after forced error\n");
    
    // Force a non-sticky error
    cudaSetDevice(0);
    cudaError_t err = cudaMalloc(nullptr, 0);  // Invalid operation
    printf("  Forced error: %s\n", cudaGetErrorString(err));
    
    // Check and clear
    recovery.checkForStickyError();
    printf("\n");
    
    printf("Sticky error recovery pattern demonstrated!\n\n");
}

// =============================================================================
// Exercise 3: Stream Error Isolation
// Handle errors in isolated streams without affecting others
// =============================================================================

class IsolatedStreamManager {
private:
    static const int MAX_STREAMS = 4;
    cudaStream_t streams[MAX_STREAMS];
    bool streamHasError[MAX_STREAMS];
    
public:
    IsolatedStreamManager() {
        for (int i = 0; i < MAX_STREAMS; i++) {
            cudaStreamCreate(&streams[i]);
            streamHasError[i] = false;
        }
    }
    
    ~IsolatedStreamManager() {
        for (int i = 0; i < MAX_STREAMS; i++) {
            cudaStreamDestroy(streams[i]);
        }
    }
    
    cudaStream_t getStream(int index) {
        if (index >= 0 && index < MAX_STREAMS) {
            return streams[index];
        }
        return nullptr;
    }
    
    bool checkStreamStatus(int index) {
        if (index < 0 || index >= MAX_STREAMS) return false;
        
        cudaError_t error = cudaStreamQuery(streams[index]);
        
        if (error == cudaSuccess) {
            printf("  Stream %d: READY (completed)\n", index);
            streamHasError[index] = false;
            return true;
        } else if (error == cudaErrorNotReady) {
            printf("  Stream %d: BUSY (still executing)\n", index);
            return true;
        } else {
            printf("  Stream %d: ERROR - %s\n", index, cudaGetErrorString(error));
            streamHasError[index] = true;
            return false;
        }
    }
    
    void synchronizeHealthyStreams() {
        printf("\n  Synchronizing healthy streams:\n");
        for (int i = 0; i < MAX_STREAMS; i++) {
            if (!streamHasError[i]) {
                cudaError_t error = cudaStreamSynchronize(streams[i]);
                if (error == cudaSuccess) {
                    printf("    Stream %d synchronized successfully\n", i);
                } else {
                    printf("    Stream %d sync failed: %s\n", i, cudaGetErrorString(error));
                    streamHasError[i] = true;
                }
            } else {
                printf("    Stream %d skipped (has error)\n", i);
            }
        }
    }
    
    int getHealthyStreamCount() {
        int count = 0;
        for (int i = 0; i < MAX_STREAMS; i++) {
            if (!streamHasError[i]) count++;
        }
        return count;
    }
};

__global__ void streamWorkKernel(float* data, int n, int streamId) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = (float)(streamId * 1000 + idx);
    }
}

void exercise3_stream_error_isolation() {
    printf("=== Exercise 3: Stream Error Isolation ===\n\n");
    
    IsolatedStreamManager manager;
    
    const int N = 1024;
    float* d_data[4];
    
    // Allocate memory for each stream
    for (int i = 0; i < 4; i++) {
        cudaMalloc(&d_data[i], N * sizeof(float));
    }
    
    printf("Launching kernels on isolated streams...\n\n");
    
    // Launch work on each stream
    for (int i = 0; i < 4; i++) {
        streamWorkKernel<<<4, 256, 0, manager.getStream(i)>>>(d_data[i], N, i);
        printf("  Launched kernel on stream %d\n", i);
    }
    
    printf("\nChecking stream status:\n");
    for (int i = 0; i < 4; i++) {
        manager.checkStreamStatus(i);
    }
    
    manager.synchronizeHealthyStreams();
    
    printf("\n  Healthy streams remaining: %d/4\n\n", manager.getHealthyStreamCount());
    
    // Cleanup
    for (int i = 0; i < 4; i++) {
        cudaFree(d_data[i]);
    }
    
    printf("Stream isolation pattern demonstrated!\n\n");
}

int main() {
    printf("╔══════════════════════════════════════════════════════════════╗\n");
    printf("║    Week 18 Day 3: Error Management - CUDA C++ Exercises      ║\n");
    printf("╚══════════════════════════════════════════════════════════════╝\n\n");
    
    exercise1_error_wrapper();
    exercise2_sticky_error_recovery();
    exercise3_stream_error_isolation();
    
    printf("All exercises completed!\n");
    return 0;
}

In [ ]:
!nvcc -O3 -arch=sm_75 -o error_management_exercises error_management_exercises.cu && ./error_management_exercises

### 🔶 Python/Numba Exercises (Optional)

The following exercises use Python with PyCUDA/Numba for those who prefer Python-based GPU programming. Error management is crucial in production Python GPU code as well.

**Python Exercise Ideas:**
1. Create a Python decorator for automatic CUDA error handling and logging
2. Implement a context manager that handles CUDA errors and provides recovery
3. Build a Python class for stream-isolated error handling with Numba

## 🎯 Key Takeaways

### Error Management Mastered! 🏨

You've learned to handle GPU emergencies like a hotel's crisis management team:

| Pattern | Description | Hotel Analogy |
|---------|-------------|---------------|
| **Always Check Errors** | Both launch and execution | Suite safety inspections |
| **cudaGetLastError()** | Clears non-sticky errors | Clear false alarms |
| **Sticky Error Reset** | cudaDeviceReset() required | Full suite renovation |
| **Stream Isolation** | One stream's error doesn't spread | Suite firewall |
| **Production Logging** | Timestamps and context | Incident reports |
| **compute-sanitizer** | Memory and race detection | Building inspector |

### 🏨 The Emergency Isolation Principle
> "With MIG, a fire in Suite A triggers local sprinklers without flooding the entire building—each partition handles its own errors independently."

### ⚡ Quick Reference
```cpp
// Check and clear non-sticky error
cudaError_t err = cudaGetLastError();
if (err != cudaSuccess) {
    log_error("Launch failed: %s", cudaGetErrorString(err));
}

// For sticky errors, reset is required
if (is_sticky_error(err)) {
    cudaDeviceReset();  // Only affects current MIG instance
}

// Stream isolation for fault containment
cudaStreamCreate(&isolated_stream);
kernel<<<grid, block, 0, isolated_stream>>>(...);
```

---

**Next:** Day 4 - Production Patterns → Running a 5-star GPU operation